## This file contains code to build a dash application. The dash app has two tabs: 
1. Table - This tab allows the user to browse through the data read using the application.
2. Chart - This tab allows user to create interactive charts by dynamically chaging values.

In [1]:
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
import pandas as pd
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import plotly.express as px

In [2]:
df = pd.read_csv('../data/demanddata-2017-2022-6H.csv')

In [4]:
# create a Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# set up pagination
PAGE_SIZE = 100

# define the layout of the app
app.layout = html.Div(children=[
    dcc.Tabs(id='tabs', value='tab-1', children=[
        dcc.Tab(label='Table', value='tab-1', children=[
            html.H3(children='UK Power Demand Data', style={'text-align': 'center'}), 
            html.Div([
                # create a table component
                dash_table.DataTable(
                    id='table',
                    columns=[{'name': i, 'id': i} for i in df.columns],
                    data=df.head(PAGE_SIZE).to_dict('records'),
                    page_size=PAGE_SIZE,
                    style_table={'overflowX': 'scroll', 'overflowY': 'scroll',
                                'height': '700px'}
                )
            ]),
            
            # add pagination controls
            dbc.Pagination(
                id='table-pagination',
                size='md',
                max_value=(len(df) // PAGE_SIZE) + 1 if len(df) % PAGE_SIZE != 0 else len(df) // PAGE_SIZE,
                fully_expanded=False,
                first_last=True,
                previous_next=True,
                style={'justify-content': 'center', "margin-top": "10px"}
            )
        ]),
        
        dcc.Tab(label='Chart', value='tab-2', children=[
            dbc.Container([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(
                            id="chart_graph",
                            figure={},
                        ),
                        
                        # add pagination controls
                        dbc.Pagination(
                            id='chart-pagination',
                            size='md',
                            max_value=(len(df) // PAGE_SIZE) + 1 if len(df) % PAGE_SIZE != 0 else len(df) // PAGE_SIZE,
                            fully_expanded=False,
                            first_last=True,
                            previous_next=True,
                            style={'justify-content': 'center', "margin-top": "25px", "color": "#800000"}
                        )
                    ], width=8),
                    dbc.Col([
                        dbc.Card([
                            dbc.CardHeader("Chart Options"),
                            dbc.CardHeader("Select chart type, group column (abscissa x-axis) and series (ordinate y-axis)", style={"fontSize": "12px"}),
                            dbc.CardBody([
                                html.Label('Chart Type'),
                                dcc.Dropdown(
                                   id="chart_type",
                                   options=[{"label": "Line", "value": "line"},
                                           {"label": "Bar", "value": "bar"}],
                                   value="line",
                                   style={"width": "100%"}
                               ),
                               html.Br(),

                               html.Label('Group Column'),
                                dcc.Dropdown(
                                    id='group_column_dropdown',
                                    options=[{'label': col, 'value': col} for col in df.columns],
                                    value=df.columns[0],
                                    style={"width": "100%"}
                                ),
                                html.Br(),
                                
                                html.Label('Series'),
                                dcc.Checklist(
                                    id="series_checklist",
                                    options=[{'label': col.lower(), 'value': col} for col in df.columns],
                                    value=[],
                                    labelStyle={'margin-right': '10px'},
                                    inputStyle={'margin-right': '16px'},
                                    style={'margin-top': '8px'}
                                ),
                                
                                html.Br(),
                                html.Button(
                                    id="add_view",
                                    n_clicks=0,
                                    children="Add View",
                                    style={
                                        "margin": "auto",
                                        "display": "block",
                                        "background-color": "#800000",
                                        "color": "white",
                                        "font-size": "16px",
                                        "padding": "7px",
                                        "textAlign": "center"
                                    }
                                )
                            ])
                        ], style={"width": "65%", "margin-top": "20px", "margin-left": "20px"}) 
                    ], width=4)
                ])
            ], fluid=True,),

        ])
        
    ])
])

# define the callback function to update the table
@app.callback(
    dash.dependencies.Output('table', 'data'),
    [dash.dependencies.Input('table-pagination', 'active_page')]
)
def update_table(active_page):
    # Set a default value for active_page if it is None
    if active_page is None:
        active_page = 1
    return df.iloc[(active_page-1)*PAGE_SIZE:(active_page)*PAGE_SIZE].to_dict('records')

# define the callback function to update the charts
@app.callback(
    Output(component_id='chart_graph', component_property='figure'),
    Input(component_id='add_view', component_property='n_clicks'),
    State(component_id='chart_type', component_property='value'),
    State(component_id='group_column_dropdown', component_property='value'),
    State(component_id='series_checklist', component_property='value'),
    Input(component_id='chart-pagination', component_property='active_page')
)

def update_chart(n_clicks, chart_type, group_column, series_columns, active_page):
    
    # Set a default value for active_page if it is None
    if active_page is None:
        active_page = 1
        
    # convert active_page to an integer
    active_page = int(active_page)
    
    # get the data for the current page
    start_index = (active_page - 1) * PAGE_SIZE
    end_index = active_page * PAGE_SIZE
    data = df.iloc[start_index:end_index]
    
    # filter the data frame based on the selected series columns
    filtered_df = data.loc[:, [group_column] + series_columns]

    # group the data frame by the group column and calculate the sum of the series columns
    grouped_df = filtered_df.groupby([group_column]).sum().reset_index()

    # create traces for each selected series column
    traces = []
    for col in series_columns:
        if chart_type == 'line':
            trace = go.Scatter(x=grouped_df[group_column], y=grouped_df[col], name=col,
                mode="lines+markers", line=dict(width=2), marker=dict(size=8),)
        else:
            trace = go.Bar(x=grouped_df[group_column], y=grouped_df[col], name=col,
                          )
        traces.append(trace)

    # create the plotly figure with the selected chart type and traces
    fig = go.Figure(data=traces)
    
    # Add layout options
    fig.update_layout(
        title=f"{', '.join(series_columns)} by {group_column}",
        xaxis=dict(title=group_column, tickmode="linear"),
        yaxis=dict(title=', '.join(series_columns)),
        legend=dict(orientation="h", yanchor="bottom", y=-0.2),
        margin=dict(l=40, r=40, t=60, b=40),
        plot_bgcolor="#f2f2f2",
        paper_bgcolor="#ffffff",
        height=750,
        width=1000,
    )

    return fig


if __name__ == '__main__':
    app.run_server(debug=True, port=8050, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
